In [ ]:
conda install -c conda-forge librosa

In [ ]:
pip  install numpy==1.20

In [ ]:
pip install --upgrade numpy==1.21.0

In [ ]:
pip install numba

In [ ]:
pip install numba

In [ ]:
import sys
print(sys.version)

In [37]:

import librosa
print(librosa.__version__)

0.8.1


In [ ]:
pip install awkward

In [1]:
import pandas as pd
import numpy as np
import pathlib
from tqdm.notebook import tqdm
import librosa   #deal with audio data
import awkward as ak  #deal with arrays with different shapes
import matplotlib.pyplot as plt

# IMPORT

In [15]:
import zipfile

zip_file_path = 'RavdessAudioOnly.zip'  # Replace with the actual path to the uploaded zip file
extract_path = 'Untitled Folder'  # Replace with the directory path where you want to extract the files

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Exclude the .ipynb_checkpoints directory
    file_list = [file for file in zip_ref.namelist() if '.ipynb_checkpoints' not in file]

    # Extract the remaining files
    for file in file_list:
        zip_ref.extract(file, extract_path)

# Optional: Remove the .ipynb_checkpoints directory
checkpoints_dir = os.path.join(extract_path, '.ipynb_checkpoints')
if os.path.exists(checkpoints_dir):
    os.rmdir(checkpoints_dir)

## LOCAL IMPORT

In [23]:
dataset_folder = pathlib.Path("Untitled Folder/RavdessAudioOnly Copia")  # change it with your file location

In [24]:
file_list = list(dataset_folder.iterdir())
for i, path in enumerate(file_list):
    if path.suffix != ".wav":
        file_list.pop(i)
file_list = sorted(file_list)
file_list #take the folder with files and put them in a list and then sorted them

[PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-01.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-02.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-03.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-04.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-05.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-06.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-07.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-08.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-09.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-10.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-11.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/03-01-01-01-01-01-12.wav'),
 PosixPath('Untitled Folder/RavdessAudioOnly Copia/0

In [25]:
array = list()
for path in tqdm(file_list):
    librosa_audio_segment, sr = librosa.load(path, sr=None)
    array.append([librosa_audio_segment]) #append to the array each audio file I read

In [26]:
b = ak.ArrayBuilder()
for path in tqdm(file_list):
    librosa_audio_segment, sr = librosa.load(path, sr=None)
    b.begin_list()
    b.begin_list()
    for value in librosa_audio_segment:
        b.real(value)
    b.end_list() 
    b.end_list() 
array = b.snapshot()   #ha lo stesso shape del dataset DM1
                        #at the end I will have a time series, where each row is a time serie

Filename identifiers 

Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
Vocal channel (01 = speech, 02 = song).
Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
Repetition (01 = 1st repetition, 02 = 2nd repetition).
Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

In [27]:
CATEGORICAL_FEATURES_NAMES = {
    "modality": {"01": "full-AV", "02": "video-only", "03": "audio-only"}, 
    "vocal_channel": {"01": "speech", "02": "song"},
    "emotion": {"01" : "neutral", "02" : "calm", "03" : "happy", "04" : "sad", "05" : "angry", "06" : "fearful", "07" : "disgust", "08" : "surprised"},
    "emotional_intensity": {"01" : "normal", "02" : "strong"},
    "statement": {"01" : "Kids are talking by the door", "02" : "Dogs are sitting by the door"},
    "repetition": {"01" : "1st", "02" : "2nd"},
    "actor": {str(i).zfill(2): str(i).zfill(2) for i in range(1, 25)}
}

#at the end I have two dataset: one with the timeseries and one with the categorical features

In [28]:
df = list()
file_names = list()
for path in file_list:
    stem = path.stem.split("-")
    file_names.append(path.name)
    df.append(stem)
df = pd.DataFrame(df)
df.columns = list(CATEGORICAL_FEATURES_NAMES.keys())

for column in df.columns:
    df = df.replace(CATEGORICAL_FEATURES_NAMES)

df["sex"] = ["F" if i % 2 == 0 else "M" for i in df["actor"].astype(int)]
df["filename"] = file_names

In [29]:
df.head()

,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,filename
0,audio-only,speech,neutral,normal,Kids are talking by the door,1st,01,M,03-01-01-01-01-01-01.wav
1,audio-only,speech,neutral,normal,Kids are talking by the door,1st,02,F,03-01-01-01-01-01-02.wav
2,audio-only,speech,neutral,normal,Kids are talking by the door,1st,03,M,03-01-01-01-01-01-03.wav
3,audio-only,speech,neutral,normal,Kids are talking by the door,1st,04,F,03-01-01-01-01-01-04.wav
4,audio-only,speech,neutral,normal,Kids are talking by the door,1st,05,M,03-01-01-01-01-01-05.wav


In [30]:
df_sorted_actor = df.sort_values(by=["actor", "filename"])
idxs_sorted = list(df_sorted_actor.index)

In [31]:
idx_train = list(df_sorted_actor[df_sorted_actor.actor.astype(int) < 19].index)
idx_test = list(df_sorted_actor[df_sorted_actor.actor.astype(int) >= 19].index)

In [32]:
X_train = array[idx_train] #there I have the time series
X_test = array[idx_test]    #there I have the time series
Y_train = df_sorted_actor[df_sorted_actor.actor.astype(int) < 19] #there I have the tabular data
Y_test = df_sorted_actor[df_sorted_actor.actor.astype(int) >= 19] #there I have the tabular data

## ONLINE IMPORT (very early stage, expect bugs!)
this will cache the dataset in your conda environment.
You should be able to find the package location with:
```
import datatime
print(datatime.__file__)
```

In [ ]:
# !pip install git+https://github.com/fspinna/datatime.git

In [ ]:
from datatime import load_dataset

In [ ]:
%%time
d = load_dataset("RavdessAudioOnly")
print(d)

In [ ]:
X_train, Y_train, X_test, Y_test = d()

In [ ]:
Y_train

# TO NUMPY

In [33]:
def pad_X(X, m_max, nan_value=0):
    return ak.fill_none(ak.pad_none(X, m_max, axis=2, clip=True), value=nan_value)

In [34]:
# find the max length of X_train
maximum = 0
for ts in X_train:
    length = len(np.asarray(np.ravel(ts)))
    if length > maximum:
        maximum = length 
maximum 

#if in the test set there is a series that have more timestamps than in the train data it will be cut

304304

In [35]:
%%time
X_train = np.squeeze(np.array(pad_X(X_train, maximum, np.nan)))
X_test = np.squeeze(np.array(pad_X(X_test, maximum, np.nan)))

#the final numpy array of train and test

CPU times: user 41.5 s, sys: 1min 53s, total: 2min 35s
Wall time: 3min 37s


In [36]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(1828, 304304)
(624, 304304)
(1828, 9)
(624, 9)


In [38]:
np.save("RavdessAudioOnlyNumpy__X_train.npy", X_train)
np.save("RavdessAudioOnlyNumpy__X_test.npy", X_test)

In [39]:
Y_train.to_csv("RavdessAudioOnlyNumpy__Y_train.csv", index=False)
Y_test.to_csv("RavdessAudioOnlyNumpy__Y_test.csv", index=False)

# FROM NUMPY

In [ ]:
X_train = np.load("RavdessAudioOnlyNumpy__X_train.npy")
X_test = np.load("RavdessAudioOnlyNumpy__X_test.npy")
print(X_train.shape, X_test.shape)

In [ ]:
X_train